# browse

> Helpers for browse, head, and tail magics
- order: 7

In [ ]:
#| default_exp browse
%load_ext autoreload
%autoreload 2

[autoreload of fastcore.basics failed: Traceback (most recent call last):
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 365, in update_class
    update_instances(old, new)
  File "c:\Users\tjhuegerich\AppData\Local\anaconda3\envs\nbstata_dev\Lib\site-packages\IPython\extensions\autoreload.py", line 323, in update_instances
    object.__setattr__(ref, "__class__", new)
TypeErr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.misc_utils import print_red
from nbstata.stata import run_single
from nbstata.stata_more import SelectVar, run_direct_cleaned, diverted_stata_output_quicker, run_sfi
from nbstata.pandas import better_pdataframe_from_data
from fastcore.basics import patch_to
import re

## Parsing Stata code for browse magic

In [ ]:
#| hide
from fastcore.test import test_eq, test_fail
from textwrap import dedent

In [ ]:
#| export
def _get_start_end_strs(stata_in_code):
    stata_range_code = stata_in_code.replace('in ','').strip()
    slash_pos = stata_range_code.find('/')
    if slash_pos != -1:
        start_str = stata_range_code[:slash_pos]
        end_str = stata_range_code[slash_pos+1:]
    else:
        start_str = "1"
        end_str = stata_range_code
    return start_str, end_str

In [ ]:
#| hide
_get_start_end_strs("in 10")

('1', '10')

In [ ]:
#| export
def _get_pos_stata_obs_num(in_obs_str, count):
    temp_str = in_obs_str.strip().upper()
    if temp_str == 'F': 
        in_obs = 1
    elif temp_str == 'L':
        in_obs = count
    else:
        try:
            in_obs = int(in_obs_str)
        except ValueError as e:
            raise ValueError(f"{in_obs_str} invalid observation number")
        if in_obs < 0: in_obs += count + 1
        if in_obs < 1 or in_obs > count:
            raise ValueError(f"{in_obs_str} invalid observation number")
    return in_obs

In [ ]:
#| hide
_get_pos_stata_obs_num("10", 100)

10

In [ ]:
#| export
def in_range(stata_in_code, count):
    """Return in-statement range"""
    if not stata_in_code.strip():
        return (None, None)
    start, end = (_get_pos_stata_obs_num(in_str, count)
                  for in_str in _get_start_end_strs(stata_in_code))
    if start > end:
        raise ValueError("observations numbers out of range")
    return (start-1, end)

In [ ]:
test_eq(in_range("", 100), (None, None))
test_eq(in_range("in 10", 100), (0, 10))
test_eq(in_range("in 1/10", 100), (0, 10))
test_eq(in_range("in F/10", 100), (0, 10))
test_eq(in_range("in 3/10", 100), (2, 10))
test_fail(in_range, args=("in 10", 5))
test_fail(in_range, args=("in 10/1", 100))

Negative indices are converted to positive, following Stata syntax:

In [ ]:
test_eq(in_range("in -10/-1", 41), (31, 41))

## Head/tail/browse magic helpers

In [ ]:
#| export
def _parse_browse_magic_syntax(code):
    _program_name = "temp_nbstata_syntax_name"
    run_direct_cleaned((
        f"program define {_program_name}\n"
        """ syntax [varlist(default=none)] [if] [in] [, noLabels noFormat]
            disp "%varlist%"
            foreach var in `varlist' {
                disp "`var'"
            }
            disp "%if%"
            disp `"`if'"'
            disp "%in%"
            disp `"`in'"'
            disp "%nolabels%"
            disp "`labels'"
            disp "%noformat%"
            disp "`format'"
            disp "%end%"
        end
        """), quietly=True)
    try:
        output = diverted_stata_output_quicker(f"""\
            {_program_name} {code}
            program drop {_program_name}
            """).strip()
    except Exception as e:
        run_sfi(f"capture program drop {_program_name}")
        raise(e)
    return output.replace("\n> ", "") #[c.strip() for c in var_code.split() if c] if var_code else None

In [ ]:
#| hide
from nbstata.config import launch_stata
from nbstata.stata import obs_count, run_single
import numpy as np

In [ ]:
#| hide
#| eval: False
launch_stata(splash=False)
run_sfi("""\
sysuse uslifeexp2, clear
replace le = . in 2/2""")

(US life expectancy, 1900–1940)
(1 real change made, 1 to missing)


In [ ]:
#| hide
#| eval: False
print(_parse_browse_magic_syntax("y if year, noformat "))

%varlist%
year
%if%
if year
%in%

%nolabels%

%noformat%
noformat
%end%


In [ ]:
#| hide
#| eval: False
print(_parse_browse_magic_syntax("y if year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year, noformat "))

%varlist%
year
%if%
if year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year & year
%in%

%nolabels%

%noformat%
noformat
%end%


In [ ]:
#| hide
#| export
matchparts = re.compile(
            r"\A.*?"
            r"^%varlist%(?P<varlist>.*?)"
            r"%if%(?P<if>.*?)"
            r"%in%(?P<in>.*?)"
            r"%nolabels%(?P<nolabels>.*?)"
            r"%noformat%(?P<noformat>.*?)%end%", #"(\Z|---+\s*end)",
            flags=re.DOTALL + re.MULTILINE).match

In [ ]:
#| hide
#| eval: False
matchparts(_parse_browse_magic_syntax("y if year, noformat ")).groupdict()

{'varlist': '\nyear\n',
 'if': '\nif year\n',
 'in': '\n\n',
 'nolabels': '\n\n',
 'noformat': '\nnoformat\n'}

In [ ]:
#| export
def parse_browse_magic(code):
    N = None
    m = re.match(r"\A(?P<N>[0-9]+)?[\s]*(?P<remainder>.*?)\Z", code.strip())
    if m is None:
        raise ValueError(f"syntax error: this magic must be in a code by itself on a single line")
    if m.group('N'):
        N = int(m.group('N'))
    code_minus_N = m.group('remainder')
    args = matchparts(_parse_browse_magic_syntax(code_minus_N)).groupdict()
    _var = [c.strip() for c in args['varlist'].split() if c]
    var = _var if _var else None
    if_code = args['if'].strip()
    in_code = args['in'].strip()
    nolabels = args['nolabels'].strip()
    noformat = args['noformat'].strip()
    return N, var, if_code, in_code, nolabels, noformat

In [ ]:
#| hide
#| eval: False
test_eq(parse_browse_magic("1, noformat"), 
        (1, None, '', '', '', 'noformat')
       )

In [ ]:
#| eval: False
code = "le if year==1920 in 1/10, noformat"
test_eq(parse_browse_magic(code), 
        (None, ['le'], 'if year==1920', 'in 1/10', '', 'noformat')
       )

In [ ]:
#| eval: False
run_single("gen date = dofy(year)")
test_eq(parse_browse_magic("10 le* if date==mdy(1, 1, 1910), noformat"),
        (10, ['le'], 'if date==mdy(1, 1, 1910)', '', '', 'noformat')
       )
run_single("drop date")

In [ ]:
#| export
def _parse_df_params(code, count, browse=False, tail=False):
    from numpy import inf
    N, var, if_code, in_code, nolabels, noformat = parse_browse_magic(code)
    sformat = not noformat
    valuelabel = not nolabels

    N_max = inf if browse else 5
    if N is not None:
        if browse:
            print_red("Warning: '%browse [N]' syntax is deprecated "
                      "and may be removed in v1.0.")
        N_max = N

    # Obs range
    obs_range = None
    if browse:
        start, end = in_range(in_code, count)
        if start != None and end != None:
            obs_range = range(start, end)
        elif count > N_max:
            obs_range = range(0, N_max)
    else:
        if in_code:
            print_red(f"Note: [in] not allowed for {'tail' if tail else 'head'} "
                      "magic and is ignored."
                     )
        if count > N_max:
            obs_range = range(count - N_max, count) if tail else range(0, N_max)

    return obs_range, var, if_code, valuelabel, sformat

In [ ]:
#| export
def get_df(obs_range, var, stata_if_code, missingval, valuelabel, sformat):
    with SelectVar(stata_if_code) as sel_varname:
        df = better_pdataframe_from_data(obs=obs_range,
                                         var=var,
                                         selectvar=sel_varname,
                                         missingval=missingval,
                                         valuelabel=valuelabel,
                                         sformat=sformat,
                                        )
        if not var and sel_varname is not None and sel_varname in df:
            df = df.drop([sel_varname], axis=1)
    return df

## Head/tail magic helpers

In [ ]:
#| export
def headtail_df_params(code, count, missing_config, tail=False):
    from numpy import nan
    custom_missingval = missing_config != 'pandas'
    missingval = missing_config if custom_missingval else nan
    obs_range, var, stata_if_code, valuelabel, sformat = (
        _parse_df_params(code, count, tail=tail)
    )
    return obs_range, var, stata_if_code, missingval, valuelabel, sformat

In [ ]:
#| eval: False
test_eq(headtail_df_params("", 50, "."),
        ((range(0, 5), None, '', ".", True, True)))

In [ ]:
#| eval: False
test_eq(headtail_df_params("10 y le*, noformat", 50, "."),
        ((range(0, 10), ['year', 'le'], '', ".", True, False)))

In [ ]:
#| eval: False
test_eq(headtail_df_params("10 y le*, noformat", 50, ".", tail=True),
        ((range(40, 50), ['year', 'le'], '', ".", True, False)))

In [ ]:
#| export
def headtail_get_df(obs_range, var, stata_if_code, missingval, valuelabel, sformat):
    if not stata_if_code:
        return get_df(obs_range, var, stata_if_code, missingval, valuelabel, sformat)
    N_max = len(obs_range)
    tail = obs_range[0] != 0
    with SelectVar(stata_if_code) as sel_varname:
        df = better_pdataframe_from_data(obs=None,
                                         var=var,
                                         selectvar=sel_varname,
                                         missingval=missingval,
                                         valuelabel=valuelabel,
                                         sformat=sformat,
                                        )
        if not var and sel_varname is not None and sel_varname in df:
            df = df.drop([sel_varname], axis=1)
    return df.tail(N_max) if tail else df.head(N_max)

In [ ]:
#| hide
#| eval: False
get_df(*headtail_df_params('', obs_count(), "."))

,year,le
1,1900,47.3
2,1901,.
3,1902,51.5
4,1903,50.5
5,1904,47.6


In [ ]:
#| hide
#| eval: False
test_eq(get_df(*headtail_df_params('', obs_count(), ".")),
        headtail_get_df(*headtail_df_params('', obs_count(), ".")),)

In [ ]:
#| eval: False
headtail_get_df(*headtail_df_params('if year<1903', obs_count(), "."))

,year,le
1,1900,47.3
2,1901,.
3,1902,51.5


In [ ]:
#| eval: False
test_eq(headtail_get_df(*headtail_df_params('if year<1903', obs_count(), ".")),
        headtail_get_df(*headtail_df_params('if year<1903', obs_count(), ".", tail=True)),)

In [ ]:
#| eval: False
headtail_get_df(*headtail_df_params('if year>1910', obs_count(), "."))

,year,le
12,1911,52.6
13,1912,53.5
14,1913,52.5
15,1914,54.2
16,1915,54.5


In [ ]:
#| hide
#| eval: False
headtail_get_df(*headtail_df_params('if year>1910', obs_count(), ".", tail=True))

,year,le
37,1936,58.5
38,1937,60
39,1938,63.5
40,1939,63.7
41,1940,62.9


In [ ]:
#| hide
#| eval: False
test_eq(headtail_get_df(*headtail_df_params('if year>1910', obs_count(), ".", tail=True)),
        headtail_get_df(*headtail_df_params('', obs_count(), ".", tail=True)),)

In [ ]:
#| hide
#| eval: False
run_single("gen date = dofy(year)")
headtail_get_df(*headtail_df_params('10 le* if date==mdy(1, 1, 1910)', obs_count(), ".", tail=False))

,le
11,50


In [ ]:
#| hide
#| eval: False
run_single("drop date")

## Browse magic helpers

In [ ]:
#| export
def browse_df_params(code, count, missing_config):
    from numpy import nan
    custom_missingval = missing_config != 'pandas'
    missingval = missing_config if custom_missingval else nan
    obs_range, var, stata_if_code, valuelabel, sformat = (
        _parse_df_params(code, count, browse=True)
    )
    return obs_range, var, stata_if_code, missingval, valuelabel, sformat

In [ ]:
#| hide
#| eval: False
test_eq(browse_df_params("y le* if year<1910 in 1/20, noformat", 50, '.'),
        (range(0, 20), ['year', 'le'], 'if year<1910', '.', True, False))

In [ ]:
#| eval: False
browse_df_params("", 10, 'pandas')

(None, None, '', nan, True, True)

In [ ]:
#| eval: False
browse_df_params("y le* if year<1910, noformat", 50, '.')

(None, ['year', 'le'], 'if year<1910', '.', True, False)

In [ ]:
#| hide
#| eval: False
browse_df_params("5 y le* if year<1910, noformat", 10, '.')

(range(0, 5), ['year', 'le'], 'if year<1910', '.', True, False)

In [ ]:
#| eval: False
browse_df_params('in 1/5', 41, '.')

(range(0, 5), None, '', '.', True, True)

In [ ]:
#| hide
#| eval: False
print(code)
get_df(*browse_df_params(code, obs_count(), '.'))

le if year==1920 in 1/10, noformat


""


In [ ]:
#| eval: False
get_df(*browse_df_params('in 1/5', obs_count(), '.'))

,year,le
1,1900,47.3
2,1901,.
3,1902,51.5
4,1903,50.5
5,1904,47.6


In [ ]:
#| export
def set_ipydatagrid_height():
    from IPython.core.display import HTML
    display(HTML("<style>div.jp-Notebook .datagrid-container {min-height: 448px; }</style>"))

In [ ]:
#| export
def display_df_as_ipydatagrid(df):
    from ipydatagrid import DataGrid, TextRenderer
    i_renderer = TextRenderer(horizontal_alignment="right", background_color="rgb(243, 243, 243)")
    d_renderer = TextRenderer(horizontal_alignment="right")
    h_renderer = TextRenderer(horizontal_alignment="center")
    column_widths = {}
    temp_head = df.head(20)
    char_px_width = 6.05
    def name_width(name_len):
        if name_len <= 15:
            return 43 + char_px_width*name_len
        elif name_len > 22 or name_len%2 == 0:
            return 43 + char_px_width*15.3
        else:
            return 43 + char_px_width*14.3
            
    for name in list(df):
        column_widths[name] = max(
            name_width(len(name)),
            20 + char_px_width*max((len(str(value)) for value in temp_head[name])),
        )
    column_widths[" "] = 20 + char_px_width*len(str(max(df.index.values)))
    g = DataGrid(df,
                 index_name=" ",
                 editable=False,
                 selection_mode='cell',
                 base_row_size=21,
                 auto_fit_columns=False,
                 default_renderer=d_renderer,
                 header_renderer=h_renderer,
                 renderers={" ": i_renderer},
                 column_widths=column_widths,
                 #layout={"height": "100%"},
                )
    g.grid_style = {
        "background_color": "rgb(255, 255, 255)",
        "header_background_color": "rgb(243, 243, 243)",
        "header_grid_line_color": "rgb(229, 229, 229)",
        "vertical_grid_line_color": "rgb(229, 229, 229)",
        "horizontal_grid_line_color": "rgb(229, 229, 229)",
        "selection_fill_color": "rgb(59, 135, 195, .25)", # 206, 225, 240
        "selection_border_color": "rgb(229, 229, 229)",
        "header_selection_fill_color": "rgb(99, 99, 99, .25)", #216
        "header_selection_border_color": "rgb(229, 229, 229)",
        "cursor_fill_color": "rgb(255, 255, 255, 0)",
        "cursor_border_color": "rgb(40, 40, 40)",
        "scroll_shadow": {'size': 0, 'color1': "white", 'color2': "white", 'color3': "white"},
    }
    display(g)

In [ ]:
#| eval: False
#set_ipydatagrid_height()
run_single('gen long_name_1234567 = ""')
run_single('gen long_str = "10000000000200000000003000000000040000000000500000000006000000000070000000000800000000009000000000010000000000"')

(41 missing values generated)


In [ ]:
#| eval: False
df = get_df(*browse_df_params(', noformat', obs_count(), '.'))
display_df_as_ipydatagrid(df)

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_row_size=21, column_widths={'ye…

In [ ]:
#| eval: False
#set_ipydatagrid_height()
run_single('sysuse auto, clear')

(1978 automobile data)


In [ ]:
#| eval: False
df = get_df(*browse_df_params('', obs_count(), '.'))
display_df_as_ipydatagrid(df)

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_row_size=21, column_widths={'ma…

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

AttributeError: module 'nbdev' has no attribute 'export'